# LeNet implementation using PyTorch

In [0]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Network definition

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))




In [0]:
batch_size = 64
gamma = 0.7
epochs = 10
test_batch_size = 1000
lr = 0.01
log_interval = 100

# Dataloader in torch

In [0]:

use_cuda = torch.cuda.is_available()

torch.manual_seed(42)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                    ])),
batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
batch_size=test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


# Training 

In [0]:
for epoch in range(1, epochs + 1):
   train(log_interval, model, device, train_loader, optimizer, epoch)
   test(model, device, test_loader)
   scheduler.step()

   torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.372624
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.535419
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.331381
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.169655
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.193956
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.259068
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.104027
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.216374
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.392459
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.303168

Test set: Average loss: 0.1717, Accuracy: 9499/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.179867
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.139794
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.304882
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.320330
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.197669
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.222902
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.161313
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.399290
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.375759
T